In [1]:
import os;
import sys;
sys.path.append("venv")

In [7]:
import pandas as pd
pd.read_parquet('../WSE_marketdata.parquet', engine='pyarrow')

,timestamp,symbol,bid_price,bid_size,ask_price,ask_size,market_period,trade_price,trade_volume,type
0,2024-01-22 05:04:15,SIM,2.78,276.0,2.850,600.0,,NaN,NaN,Quote
1,2024-01-22 08:30:30,SIM,2.79,1000.0,2.850,600.0,COCA,NaN,NaN,Quote
2,2024-01-22 09:01:02,SIM,2.79,1000.0,2.820,36.0,COCO,NaN,NaN,Quote
3,2024-01-22 09:40:30,SIM,2.79,1000.0,2.820,31.0,COCO,NaN,NaN,Quote
4,2024-01-22 09:40:30,SIM,NaN,NaN,NaN,NaN,None,2.82,5.0,Trade
...,...,...,...,...,...,...,...,...,...,...
645580,2024-01-22 16:36:15,RES,0.57,1000.0,0.598,5000.0,COCO,NaN,NaN,Quote
645581,2024-01-22 16:36:15,RES,NaN,NaN,NaN,NaN,None,0.59,4500.0,Trade
645582,2024-01-22 16:37:38,RES,0.57,1000.0,0.592,1300.0,COCO,NaN,NaN,Quote
645583,2024-01-22 16:43:08,RES,0.57,1000.0,0.590,4500.0,COCO,NaN,NaN,Quote


In [10]:


dir(aggregator)

['Aggregate',
 'Aggregator',
 'Filter',
 'NamedTuple',
 'NoopFilter',
 'Quote',
 'Trade',
 'Type',
 'WallClockSliceSchedule',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'abstractmethod',
 'namedtuple',
 'pd',
 'time',
 'timedelta']

In [11]:
import sys
import aggregator
import filters


class volumeAggregate(aggregator.Aggregate):
    def __init__(self):
        self.volume = 0

    def on_quote(self, quote):
        pass

    def on_trade(self, trade):
        self.volume += trade.trade_volume

    def compute_slice(self, slice):
        ret = self.volume
        self.volume = 0
        return ret


agg = aggregator.Aggregator("../WSE_marketdata.parquet","../WSE_symbology.parquet", filters.SymbolFilter('PKO'))
agg.registerAggregate(volumeAggregate)
agg.run()
dir(agg)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'aggregate_values',
 'aggregates',
 'aggs_to_dataframe',
 'filter',
 'market_data_path',
 'on_slice_triggered',
 'readSymbology',
 'registerAggregate',
 'run',
 'slice_schedule',
 'symbology']